In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [14]:
import pandas as pd
import lxml as lxml
import io as io
from concurrent.futures import ThreadPoolExecutor

from script import request_tri
from script import process_data

In [3]:
# Charger le fichier Parquet contenant les transactions
transactions = pd.read_parquet("data/base.parquet", engine="pyarrow")

In [4]:
# Préparer les données nécessaires (coordonnées)
coordinates = list(zip(transactions['latitude'], transactions['longitude']))

# Limiter le nombre de threads simultanés pour éviter la surcharge
with ThreadPoolExecutor(max_workers=5) as executor:  # Limite de 5 threads
    results = list(executor.map(request_tri.check_inondable_parallel, coordinates))

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results, columns=['results', 'identifiant_tri', 'libelle_type_inondation', 'code_scenario'])

# Fusionner les résultats avec les données originales
transactions = pd.concat([transactions.reset_index(drop=True), results_df], axis=1)

In [5]:
# Sauvegarder le fichier avec les résultats
transactions = transactions.rename(columns={'results': 'zone_inondable'})
transactions['zone_inondable'] = transactions['zone_inondable'].replace({2: 0, 3: 0})

transactions.to_parquet("data/transactions_with_zone_inondable.parquet", index=False, engine="pyarrow")


La requête de l'API Géorisques renseigne 4 informatons :
- la présence ou non dans une zone inondable (results)
- si présence, la zone géographique TRI (Territoire à Risque Important d'Inondation) à laquelle le point géographique est associé 
- si présence, le code scénario associé. Il existe 4 scénarios distincts :
    - l'aléa de forte probabilité (01For) dénommé "évènement fréquent" avec une période de retour de 10 à 30 ans
    - l'aléa de moyenne probabilité (02Moy) dénommé "évènement moyen", avec une période d eretour de 100 à 300 ans
    - l'aléa de moyenne probabilité avec changement climatique (03Mcc) dénommé "évènement moyen avec changement climatique" (qui est une majoration d'un évènement moyen)
    - l'aléa de faible probabilité (03Fai), dénommé "évènement extrême" avec une période de retour d'au moins 1000 ans

La période de retour est la durée moyenne au cours de laquelle un évènement d'une même intensité est amené à se reproduire

- si présence, l'aléa d'inondation associé : submersion marine, débordements des cours d'eau, ruissellement et débordements des eaux souterraines

In [6]:
# Étape 1 : Compter et afficher le nombre de transactions par commune
transaction_counts = transactions.groupby('nom_commune').size().sort_values(ascending=False)
print(transaction_counts)

nom_commune
Nice           7721
Nantes         5048
Bordeaux       4601
Toulon         3075
Cannes         2831
               ... 
Ogliastro         1
Mauny             1
Locmaria          1
Cagnano           1
Île-de-Batz       1
Length: 946, dtype: int64


In [7]:
# Filtrer les transactions pour les Maisons
filtered_maisons = transactions[transactions['type_local'] == 'Maison']
filtered_maisons = filtered_maisons.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['zone_inondable'].sum() > 5 and (group['zone_inondable'] == 0).any()
)

# Filtrer les transactions pour les Appartements
filtered_appartements = transactions[transactions['type_local'] == 'Appartement']
filtered_appartements = filtered_appartements.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['zone_inondable'].sum() > 5 and (group['zone_inondable'] == 0).any()
)

# Combiner les deux sous-ensembles filtrés
filtered_transactions = pd.concat([filtered_maisons, filtered_appartements])


In [8]:
# Étape 3 : Calculer la moyenne agrégée des prix par commune et par zone inondable
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

In [9]:
# Moyenne par commune et zone inondable
mean_prices = filtered_transactions.groupby(['nom_commune', 'zone_inondable'])['prix_m2'].mean().unstack()

On compare les prix moyens en zone inondable. Pour les appartements, les écarts sont à première vue importants mais ne vont pas dans le même sens selon les communes

In [15]:
# Appeler la fonction mise à jour
appart_table = process_data.produce_stats(filtered_appartements, 'data/moyenne_appartements.csv')

# Afficher le tableau dans le notebook
display(appart_table)

nom_commune,Population,prix_moyen_non_inondable,prix_moyen_inondable,écart,nb_transactions,part_inondable
Nice,348 085,5 030,4 433,-11.9%,7 476,13.0%
Nantes,323 204,3 705,3 911,5.6%,4 100,11.7%
Bordeaux,261 804,4 931,3 925,-20.4%,3 510,8.3%
Toulon,180 452,3 034,2 456,-19.0%,2 648,22.2%
Perpignan,119 656,1 626,1 597,-1.8%,1 895,25.3%
Rouen,114 083,2 811,2 616,-6.9%,1 933,18.1%
Caen,108 200,2 912,3 270,12.3%,1 725,2.3%
Dunkerque,86 788,2 151,1 992,-7.4%,495,12.3%
Béziers,80 341,1 614,1 334,-17.3%,1 020,2.6%
Cherbourg-en-Cotentin,77 808,2 169,2 331,7.4%,357,20.2%


Pour les maisons, on fait un constat similaire. Il n'y a, à première vue, pas de différence marquée, au global entre le prix des maisons en zone inondable et celles en zone non-inondables

In [16]:

# Processus pour les maisons
maison_table = process_data.produce_stats(filtered_maisons, 'data/moyenne_maisons.csv')
display(maison_table)  # Affiche dans le notebook


nom_commune,Population,prix_moyen_non_inondable,prix_moyen_inondable,écart,nb_transactions,part_inondable
Nice,348 085,7 189,16 753,133.0%,245,4.9%
Nantes,323 204,4 767,4 932,3.5%,948,2.3%
Bordeaux,261 804,5 696,5 208,-8.6%,1 091,3.7%
Toulon,180 452,4 813,4 091,-15.0%,427,10.8%
Perpignan,119 656,2 327,2 075,-10.8%,546,18.3%
Rouen,114 083,3 072,2 904,-5.5%,367,6.8%
Dunkerque,86 788,1 989,2 215,11.4%,606,11.2%
Béziers,80 341,2 476,2 416,-2.4%,473,4.0%
Cherbourg-en-Cotentin,77 808,2 402,2 389,-0.6%,567,10.6%
Saint-Nazaire,72 057,3 244,2 406,-25.8%,559,8.8%
